In [33]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense

# Set up directory and parameters
base_dir = 'Dataset/'
img_size = 180  # Ensured to use 180 as the image size
batch_size = 32
image_size = (img_size, img_size)

# Count total images in dataset
count = 0
dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

for dir in dirs:
    files = list(os.listdir(f'{base_dir}/{dir}'))
    print(f"{dir} Folder has {len(files)} files")
    count += len(files)

print(f"Total Images in Dataset Folder: {count}")

# Create training and validation datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.4,
    subset='training',
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.4,
    subset='validation',
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

class_names = train_ds.class_names
print("Class names:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Define the ResNet model
resnet_model = Sequential()

# Load the pre-trained ResNet50 model
pretrained_model = ResNet50(
    include_top=False,
    input_shape=(img_size, img_size, 3),
    pooling='avg',  # Global average pooling
    weights='imagenet'
)

# Freeze the layers of the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add layers to the model
resnet_model.add(pretrained_model)
resnet_model.add(Dense(len(dirs), activation='softmax'))  # Adjust number of classes

# Build the model
resnet_model.build((None, img_size, img_size, 3))  # Set the input shape

# Compile the model
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Add early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
epochs = 20
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)



LEAF FOLDERS Folder has 1020 files
STEM BORER Folder has 1000 files
RICE BUG Folder has 1007 files
Total Images in Dataset Folder: 3027
Found 3027 files belonging to 3 classes.
Using 1817 files for training.
Found 3027 files belonging to 3 classes.
Using 1210 files for validation.
Class names: ['LEAF FOLDERS', 'RICE BUG', 'STEM BORER']
Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.4263 - loss: 1.2400 - val_accuracy: 0.7314 - val_loss: 0.6773
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.7532 - loss: 0.6438 - val_accuracy: 0.8471 - val_loss: 0.4650
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.8330 - loss: 0.4705 - val_accuracy: 0.8876 - val_loss: 0.3750
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.8780 - loss: 0.3703 - val_accuracy: 0.9000 - val_loss: 0.3160
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.9100 - loss: 0.3147 - val_accuracy: 0.9149 - val_loss: 0.2765
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━

In [34]:
resnet_model.save('my_trained_model2.keras')  # Save the model



In [48]:
import os
import numpy as np
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('my_trained_model.keras')

# Define the class names
rice_pestnames = ['LEAF FOLDER', 'RICE BUG', 'STEM BORER']  # Ensure the order matches your training classes

def classify_image(image_path):
    # Load and preprocess the image
    input_image = tf.keras.utils.load_img(image_path, target_size=(180, 180))  # Match your training image size
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array, 0)  # Add batch dimension

    # Make predictions
    predictions = model.predict(input_image_exp_dim)

    # Get the predicted class and confidence
    predicted_class = rice_pestnames[np.argmax(predictions)]  # Use 'predictions' variable
    confidence = np.max(predictions) * 100  # Convert to percentage

    print(f"The image belongs to '{predicted_class}' with a confidence level of {confidence:.2f}%")

# Test with an example image
image_path = 'rr.jpg'  # Replace with the path to your test image
classify_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 699ms/step
The image belongs to 'RICE BUG' with a confidence level of 97.98%
